# For the Human Touch
Human-In-The-Loop implementation of the damage detection task.

In [1]:
import gradio as gr
import cv2
import numpy as np
import os
import glob
import numpy as np
import matplotlib.pyplot as plt

In [4]:
damaged_image = "../../data/predicted_original.png"
mask = "../../data/predicted_mask.png"

def overlay_mask_on_image(image_path, mask_path, alpha=0.5):
    if not os.path.exists(image_path) or not os.path.exists(mask_path):
        raise FileNotFoundError("One or both image paths are invalid.")
    
    image = cv2.imread(image_path)
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

    # Resize mask to match image if needed
    if mask.shape != image.shape[:2]:
        mask = cv2.resize(mask, (image.shape[1], image.shape[0]))

    # Make overlay: red where mask = 255
    overlay = image.copy()
    overlay[mask > 0] = [0, 0, 255]  # red

    # Blend with alpha
    blended = cv2.addWeighted(image, 1 - alpha, overlay, alpha, 0)
    return cv2.cvtColor(blended, cv2.COLOR_BGR2RGB)

def save_edited_mask(edited_data):
    # Load original predicted mask (ensure it's 8-bit grayscale 0/255)
    orig_mask = cv2.imread(mask, cv2.IMREAD_GRAYSCALE)
    if orig_mask is None:
        raise FileNotFoundError("unet_mask.png not found")

    if isinstance(edited_data, dict) and "composite" in edited_data:
        composite = edited_data["composite"]
        background = edited_data["background"]

        # Downscale to match original mask size
        target_size = (orig_mask.shape[1], orig_mask.shape[0])
        composite = cv2.resize(composite, target_size, interpolation=cv2.INTER_NEAREST)
        background = cv2.resize(background, target_size, interpolation=cv2.INTER_NEAREST)

        # Convert both to grayscale
        composite_gray = cv2.cvtColor(composite, cv2.COLOR_RGB2GRAY)
        background_gray = cv2.cvtColor(background, cv2.COLOR_RGB2GRAY)

        # Extract user corrections
        diff = cv2.absdiff(composite_gray, background_gray)
        _, user_mask = cv2.threshold(diff, 20, 255, cv2.THRESH_BINARY)

        # Ensure original mask is binary
        _, orig_mask_bin = cv2.threshold(orig_mask, 127, 255, cv2.THRESH_BINARY)

        # Combine original mask + user corrections
        user_mask = cv2.resize(user_mask, (orig_mask_bin.shape[1], orig_mask_bin.shape[0]), interpolation=cv2.INTER_NEAREST)
        final_mask = cv2.bitwise_or(orig_mask_bin, user_mask)

        # Save and return for preview
        cv2.imwrite("mask_corrected.png", final_mask)
        return cv2.cvtColor(final_mask, cv2.COLOR_GRAY2RGB)

    return np.zeros((320, 320, 3), dtype=np.uint8)


# Generate overlay preview
# composite_img = overlay_mask_on_image("predicted_original.png", "predicted_mask.png")

composite_img = overlay_mask_on_image(damaged_image, mask)
composite_img_upscaled = cv2.resize(composite_img, None, fx=4, fy=4, interpolation=cv2.INTER_LINEAR)

with gr.Blocks() as demo:
    gr.Markdown("### Trace over missing cracks in the predicted mask")
    editor = gr.ImageEditor(label="Draw on Mask Overlay", value=composite_img_upscaled)
    save_btn = gr.Button("Save Refined Mask")
    output = gr.Image(label="Saved Mask Preview")

    save_btn.click(fn=save_edited_mask, inputs=editor, outputs=output)

demo.launch(share=True)


* Running on local URL:  http://127.0.0.1:7871
* Running on public URL: https://6adc6064d706fa9ba9.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
